In [2]:
from langchain_ollama import OllamaLLM
from langchain.chat_models import init_chat_model
import getpass
import os
import torch

llm = init_chat_model("llama3.1",model_provider = "Ollama")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [3]:

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")
from langchain_tavily import TavilySearch
search_tool = TavilySearch(
    max_results=5,
    topic="general"
)

In [ ]:
""" from langgraph.prebuilt import create_react_agent
agent = create_react_agent(llm, [tool])
user_input = "When is jee advanced 2025 going to be held?"
for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print() """

' from langgraph.prebuilt import create_react_agent\nagent = create_react_agent(llm, [tool])\nuser_input = "When is jee advanced 2025 going to be held?"\nfor step in agent.stream(\n    {"messages": user_input},\n    stream_mode="values",\n):\n    step["messages"][-1].pretty_print() '

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from accelerate import init_empty_weights
embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
response = search_tool.invoke({"query": "When is jee advanced 2025 going to be held?"})
raw_docs = response['results']
docs = [
    Document(page_content=res['content'],metadata={"url": res['url']}) for res in raw_docs
]
db = Chroma.from_documents(docs,embed,persist_directory="chroma_db")
db


In [8]:
from langchain_core.prompts import ChatPromptTemplate

template = """
You are an AI agent, specifically modeled to answer questions related to various engineering enterance exams in India.
Here is some relevant context : {context}
Here is the query : {query}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
document_chain = create_stuff_documents_chain(llm,prompt)
retiever = db.as_retriever()
retiever
retrieval_chain = create_retrieval_chain(retiever,document_chain)
retrieval_chain.invoke({"context": "JEE Advanced"})

KeyError: "Input to ChatPromptTemplate is missing variables {'query'}.  Expected: ['context', 'query'] Received: ['input', 'context']\nNote: if you intended {query} to be part of the string and not a variable, please escape it with double curly braces like: '{{query}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "